In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
from datetime import datetime
import traceback

In [5]:
ruta = "C:/Users/9 ----- SIG/Downloads/RPC_067.xlsx"
df = pd.read_excel(ruta)

In [ ]:
def seleccionar_motivo_con_reintentos(driver, max_intentos=1):
    # Esperar que el dropdown esté completamente cargado
    dropdown_justificacion = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, "//span[contains(@class, 'ui-icon-triangle-1-s')]"))
    )
    
    # Hacer scroll hacia el elemento para asegurar que esté visible
    driver.execute_script("arguments[0].scrollIntoView(true);", dropdown_justificacion)
    time.sleep(0.5)
    
    dropdown_justificacion.click()
    print(f"✅ Dropdown clickeado")
    time.sleep(0.7)  # Tiempo extra para que se abra completamente
    
    # Verificar que las opciones estén disponibles
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "formDeleteAll:justificacion3_2"))
    )
    
    # Seleccionar "Error de reporte"
    print(f"🔍 Seleccionando 'Error de reporte'...")
    opcion_error_reporte = driver.find_element(By.ID, "formDeleteAll:justificacion3_2")
    driver.execute_script("arguments[0].click();", opcion_error_reporte)
    print(f"✅ Opción 'Error de reporte' seleccionada")
    time.sleep(0.4)

# Crear el navegador
driver = webdriver.Chrome()

try:
    # Abrir la página de inicio de sesión
    driver.get("https://hecaa.mineducacion.gov.co/hecaa-snies/content/admin/login/login.jsf")
    time.sleep(0.5)
    
    # Ingresar usuario y contraseña
    usuario = driver.find_element(By.CSS_SELECTOR, "input[id='loginForm:login-user']")
    usuario.send_keys("1826-admin")
    contrasena = driver.find_element(By.CSS_SELECTOR, "input[id='loginForm:login-password']")
    contrasena.send_keys("1826Uan9123")
    
    # Clic en el botón de ingresar
    boton_ingresar = driver.find_element(By.ID, "loginForm:submitLogin")
    boton_ingresar.click()
    
    # Ir a la sección MDM -> Novedad Poblacional
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "menu_129"))).click()
    time.sleep(0.5)
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "menu_135"))).click()
    time.sleep(0.5)
    
    # Seleccionar tipo de documento UNA SOLA VEZ (Cédula de ciudadanía)
    dropdown = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "filter:tipoDoc_label")))
    dropdown.click()
    time.sleep(0.4)
    opcion = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//li[@data-label='Cédula de ciudadanía']")))
    opcion.click()
    time.sleep(0.4)
    
    resultados = []  # informe al final del bucle
    
    # Procesar cada registro del DataFrame
    for i in range(len(df)):
        try:
            # Ingresar número de cédula
            cedula = str(df['NUM_DOCUMENTO'].iloc[i])
            input_cedula = driver.find_element(By.ID, "filter:numDoc")
            input_cedula.clear()
            input_cedula.send_keys(cedula)
            time.sleep(0.3)
            
            # Buscar
            boton_buscar = WebDriverWait(driver, 4).until(EC.element_to_be_clickable((By.XPATH, "//span[text()='Buscar']")))
            boton_buscar.click()
            time.sleep(0.5)
            
            # Clic en "Consultar"
            codigo_programa = str(df['CODIGO_PROGRAMA'].iloc[i])
            fila_codigo = WebDriverWait(driver, 4).until(
                EC.presence_of_element_located((By.XPATH, f"//td[contains(text(), '{codigo_programa}')]/parent::tr"))
            )
            boton_consultar = fila_codigo.find_element(By.TAG_NAME, "a")
            boton_consultar.click()
            time.sleep(0.3)

            # Eliminar información
            boton_eliminar = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//span[contains(text(), 'Eliminar Información Poblacional')]"))
            )
            boton_eliminar.click()
            time.sleep(0.5)

            # Seleccionar motivo con reintentos
            seleccionar_motivo_con_reintentos(driver)
            # Continuar con el botón de eliminar
            print(f"🔍 Buscando botón 'Eliminar Información'...")
            boton_eliminar_info = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.ID, "formDeleteAll:j_idt182"))
            )          
            try:
                    boton_eliminar_info.click()
                    print(f"✅ Botón 'Eliminar Información' clickeado")
            except:
                    print(f"⚠️ Click interceptado, usando JavaScript...")
                    driver.execute_script("arguments[0].click();", boton_eliminar_info)
                    print(f"✅ Botón 'Eliminar Información' clickeado con JavaScript")
                
                    time.sleep(5)
            
            # Clic en "Volver"
            boton_volver = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//span[contains(@class, 'ui-button-text') and contains(text(), 'Volver')]"))
            )
            boton_volver.click()
            time.sleep(0.3)
            
            print(f"✅ Registro {i + 1} procesado con éxito (Cédula: {cedula})")
            resultados.append({
                "CÉDULA": cedula,
                "CÓDIGO_PROGRAMA": codigo_programa,
                "ESTADO": "Procesado con éxito"
            })
            
        except Exception as e:
            print(f"❌ Error en el registro {i + 1} (Cédula: {cedula}): {e}")
            # Si falla, intentar volver a la pantalla de búsqueda
            try:
                boton_volver = WebDriverWait(driver, 3).until(
                    EC.element_to_be_clickable((By.XPATH, "//span[contains(@class, 'ui-button-text') and contains(text(), 'Volver')]"))
                )
                boton_volver.click()
                time.sleep(0.3)
            except:
                pass
            
            # Guardar como error
            resultados.append({
                "CÉDULA": cedula,
                "CÓDIGO_PROGRAMA": df['CODIGO_PROGRAMA'].iloc[i],
                "ESTADO": f"Error: {str(e).splitlines()[0]}"
            })

finally:
    # Cerrar navegador al final
    driver.quit()
    
    # Guardar resultados
    df_resultados = pd.DataFrame(resultados)
    df_resultados.to_excel("resultado_procesamiento.xlsx", index=False)
    print("📄 Archivo 'resultado_procesamiento.xlsx' generado con los resultados.")